In [1]:
%run ../00_AdvancedPythonConcepts/talktools.py

# Web Development with Python

AY250 Spring 2018

```
git pull; pip install flask cherrypy
```

<img src="http://www.opensourceforu.com/wp-content/uploads/temp-uploads/2010/06/Python-bite.jpg">

http://www.linuxforu.com/how-to/django-when-python-bites-the-web/

<img src="imgs/droppedimage.png">

## Overview 

1. The web paradigm

2. Using Python for the Web

   - Basic Python servers

   - Frameworks and using Flask


## Web Interactions

We already saw this in the Interaction Lecture:


<img src="https://docstore.mik.ua/orelly/apache_mod/03fig01.gif">



 https://www.khanacademy.org/computing/computer-programming/html-css
 <img src="imgs/khan.png">

## Browser stuff

<img src="imgs/browser.png">

- Python helps generate the (html) content.
- In general, you work on CSS and JS separately.
- Finally, use Python to serve all of this media to the user.


`http` is a package that collects several modules for working with the HyperText Transfer Protocol:

  - http.client is a low-level HTTP protocol client; for high-level URL opening use urllib.request
  - http.server contains basic HTTP server classes based on socketserver
  - http.cookies has utilities for implementing state management with cookies
  - http.cookiejar provides persistence of cookies

## Simple Servers

Recall the XML/RPC Server...a webserver just responds to a different request protocol

In [2]:
!python3 -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [09/Apr/2018 14:25:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2018 14:25:18] "GET /app.py HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2018 14:25:39] "GET /fhello.py HTTP/1.1" 200 -
^C

Keyboard interrupt received, exiting.


In [9]:
%%writefile hello.py

from http.server import BaseHTTPRequestHandler, HTTPServer

HOST = "localhost"
PORT = 8083

class Simpleserver(BaseHTTPRequestHandler):
   
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(bytes("<body>Hello!</body>", "utf-8"))

myServer = HTTPServer((HOST, PORT), Simpleserver)

try:
    myServer.serve_forever()
except KeyboardInterrupt:
    pass

myServer.server_close()

Overwriting hello.py


In [10]:
%run hello.py

127.0.0.1 - - [09/Apr/2018 14:29:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2018 14:29:47] "GET /favicon.ico HTTP/1.1" 200 -


**note: HTTPServer is not threaded**

In [11]:
%%writefile httpd.py
import time
import http.server

HOST_NAME = 'localhost' #
PORT_NUMBER = 8091 # Maybe set this to 9000.

def s2b(s):
    return bytes(s, 'utf-8')

class MyHandler(http.server.SimpleHTTPRequestHandler):
    def do_HEAD(s):
        s.send_response(200)
        s.send_header(s2b("Content-type"), s2b("text/html"))
        s.end_headers()

    def do_GET(s):
        """Respond to a GET request."""
        s.send_response(200)
        s.send_header(s2b("Content-type"), s2b("text/html"))
        s.end_headers()
        s.wfile.write(s2b("<html><head><title>Title goes here.</title></head>"))
        s.wfile.write(s2b("<body><p>This is a <i>test.</i></p>"))
        # If someone went to "http://something.somewhere.net/foo/bar/",
        # then s.path equals "/foo/bar/".
        s.wfile.write(s2b("<p>You accessed path: %s</p>" % s.path))
        s.wfile.write(s2b("</body></html>"))

if __name__ == '__main__':
    server_class = http.server.HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime()), "Server Starts - %s:%s" % (HOST_NAME, PORT_NUMBER)
    httpd.serve_forever()
    #except KeyboardInterrupt:
    #    pass
    httpd.server_close()
    print(time.asctime()), "Server Stops - %s:%s" % (HOST_NAME, PORT_NUMBER)

Overwriting httpd.py


In [12]:
%run httpd.py

Mon Apr  9 14:34:20 2018


127.0.0.1 - - [09/Apr/2018 14:34:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2018 14:34:25] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2018 14:34:34] "GET /hello/class HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2018 14:34:34] "GET /favicon.ico HTTP/1.1" 200 -


KeyboardInterrupt: 

### Cherrypy 

<img width="100" src="http://docs.cherrypy.org/en/latest/_static/images/cherrypy_logo_big.png" >

http://cherrypy.org/: "pythonic, object-oriented web framework"
```
pip install cherrypy
```

In [15]:
%%writefile cp1.py
import cherrypy

PORTNUM = 8098

class WelcomePage:
    def greetUser(self, name = None):
        if name:
            # Greet the user!
            return "Hey %s, what's up?" % name
        else:
            return 'call me like <i>http://localhost:{}/greetUser?name=Josh</i>'.format(PORTNUM)
    
    greetUser.exposed = True

cherrypy.config.update({"server.socket_port": PORTNUM})

cherrypy.quickstart(WelcomePage())

Overwriting cp1.py


In [14]:
%run cp1.py

[09/Apr/2018:14:36:34] ENGINE Listening for SIGTERM.
[09/Apr/2018:14:36:34] ENGINE Listening for SIGHUP.
[09/Apr/2018:14:36:34] ENGINE Listening for SIGUSR1.
[09/Apr/2018:14:36:34] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[09/Apr/2018:14:36:34] ENGINE Started monitor thread 'Autoreloader'.
[09/Apr/2018:14:36:34] ENGINE Serving on http://127.0.0.1:8093
[09/Apr/2018:14:36:34] ENGINE Bus STARTED


127.0.0.1 - - [09/Apr/2018:14:36:48] "GET / HTTP/1.1" 404 1394 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
127.0.0.1 - - [09/Apr/2018:14:37:01] "GET /greetUser HTTP/1.1" 200 61 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
127.0.0.1 - - [09/Apr/2018:14:37:14] "GET /greetUser?name=Josh HTTP/1.1" 200 20 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
127.0.0.1 - - [09/Apr/2018:14:37:20] "GET /greetUser?name=John HTTP/1.1" 200 20 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"


[09/Apr/2018:14:38:24] ENGINE Keyboard Interrupt: shutting down bus
[09/Apr/2018:14:38:24] ENGINE Bus STOPPING
[09/Apr/2018:14:38:24] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8093)) shut down
[09/Apr/2018:14:38:24] ENGINE Stopped thread 'Autoreloader'.
[09/Apr/2018:14:38:24] ENGINE Bus STOPPED
[09/Apr/2018:14:38:24] ENGINE Bus EXITING
[09/Apr/2018:14:38:24] ENGINE Bus EXITED
[09/Apr/2018:14:38:24] ENGINE Waiting for child threads to terminate...


In [2]:
%%writefile cp2.py
"""
Tutorial - Passing variables

This tutorial shows you how to pass GET/POST variables to methods.
"""
import os
import cherrypy

class WelcomePage:

    @cherrypy.expose
    def index(self):
        # Ask for the user's name.
        return '''
            <form action="greetUser" method="GET">
            What is your name?
            <input type="text" name="name" />
            What is your favorite color?
            <input type="text" name="color" />
            <input type="submit" />
            </form>'''
    
    @cherrypy.expose
    def greetUser(self, name = None, color = "blue"):
        # CherryPy passes all GET and POST variables as method parameters.
        # It doesn't make a difference where the variables come from, how
        # large their contents are, and so on.
        #
        # You can define default parameter values as usual. In this
        # example, the "name" parameter defaults to None so we can check
        # if a name was actually specified.
        
        if name:
            # Greet the user!
            return "<font color='%s'>Hey %s, what's up?</font>" % (color, name)
        else:
            if name is None:
                # No name was specified
                return 'Please enter your name <a href="./">here</a>.'
            else:
                return 'No, really, enter your name <a href="./">here</a>.'
        
    def secret(self):
        return "here's my password: blah!"

if __name__ == '__main__':
    # CherryPy always starts with app.root when trying to map request URIs
    # to objects, so we need to mount a request handler root. A request
    # to '/' will be mapped to HelloWorld().index().
    cherrypy.config.update({"server.socket_port": 8083})

    cherrypy.quickstart(WelcomePage())
else:
    # This branch is for the test suite; you can ignore it.
    cherrypy.tree.mount(WelcomePage())


Overwriting cp2.py


In [3]:
%run cp2.py

[09/Apr/2018:14:55:39] ENGINE Listening for SIGTERM.
[09/Apr/2018:14:55:39] ENGINE Listening for SIGHUP.
[09/Apr/2018:14:55:39] ENGINE Listening for SIGUSR1.
[09/Apr/2018:14:55:39] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[09/Apr/2018:14:55:39] ENGINE Started monitor thread 'Autoreloader'.


127.0.0.1 - - [09/Apr/2018:14:55:39] "GET /greetUser?name=teststest HTTP/1.1" 200 51 "https://tough-pig-50.localtunnel.me/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"


[09/Apr/2018:14:55:39] ENGINE Serving on http://127.0.0.1:8083
[09/Apr/2018:14:55:39] ENGINE Bus STARTED


127.0.0.1 - - [09/Apr/2018:14:55:40] "GET / HTTP/1.1" 200 272 "" "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:59.0) Gecko/20100101 Firefox/59.0"
127.0.0.1 - - [09/Apr/2018:14:55:40] "GET /greetUser?name=teststest HTTP/1.1" 200 51 "https://tough-pig-50.localtunnel.me/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
127.0.0.1 - - [09/Apr/2018:14:55:40] "GET /greetUser?name=josh HTTP/1.1" 200 46 "https://tough-pig-50.localtunnel.me/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15"
127.0.0.1 - - [09/Apr/2018:14:55:40] "GET /greetUser?name=brooooo HTTP/1.1" 200 49 "https://tough-pig-50.localtunnel.me/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
127.0.0.1 - - [09/Apr/2018:14:55:40] "GET /greetUser?name=josh HTTP/1.1" 200 46 "https://tough-pig-50.localtunnel.me/" "Moz

[09/Apr/2018:14:57:18] ENGINE Keyboard Interrupt: shutting down bus
[09/Apr/2018:14:57:18] ENGINE Bus STOPPING
[09/Apr/2018:14:57:23] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8083)) shut down
[09/Apr/2018:14:57:23] ENGINE Stopped thread 'Autoreloader'.
[09/Apr/2018:14:57:23] ENGINE Bus STOPPED
[09/Apr/2018:14:57:23] ENGINE Bus EXITING
[09/Apr/2018:14:57:23] ENGINE Bus EXITED
[09/Apr/2018:14:57:23] ENGINE Waiting for child threads to terminate...
[09/Apr/2018:14:57:23] ENGINE Waiting for thread CP Server Thread-26.


In [ ]:
https://tough-pig-50.localtunnel.me

You can share your running webserver using `localtunnel` (https://localtunnel.github.io/www/)

```
brew install npm # on mac
npm install -g localtunnel
lt --port 8083
```

# Small exercise:

modify cp2.py so that it asks the user for their name and their favorite color. Then greet them with that color...

hint: `<font color="red">output</font>`

<img src="imgs/form1.png">

<img src="imgs/form2.png">
